# **LAB-05 REDES NEURONALES - REGRESIÓN LOGÍSTICA MULTICLASE** 

### Nombre: Gonzales Suyo Franz Reinaldo

### Carrera: Ing. de Sistemas

### C.U. 35-5335

# Implementación del modelo de regresión logística multiclase con una Red Neuronal

En este ejercicio implementaremos una red neuronal con regresion logistica multiclase y se aplica a dos diferentes datasets.

Nuestro objetuvo es predecir los número de label que son escritos del 0 al 9 en una imagen de 28x28 pixeles.
En el siguiente dataset **MNIST FGSM** `minist_train.csv` se encuentran todos los datos.

Link del Dataset: https://www.kaggle.com/datasets/sudulakishore/mnist-fgsm?select=mnist_train.csv

Link del Repositorio de GitHub LAB-04: https://github.com/Gonzales-Franz-Reinaldo/SIS420-AI/tree/main/Laboratorios/LAB-05

# **1. Preprocesamiento de los Datos**

## Información del Dataset

### MNIST FGSM

Un conjunto de datos similar al MNIST de 70.000 imágenes de 28 x 28 etiquetadas como Método de señalización de gradiente rápido


### Acerca del conjunto de datos

#### Contexto

MNIST-FGSM es un conjunto de datos de imágenes adversarias que consta de un conjunto de entrenamiento de 60.000 ejemplos y un conjunto de prueba de 10.000 ejemplos. Cada ejemplo es una imagen en escala de grises de 28x28, asociada a una etiqueta de 10 clases. MNIST-FGSM tiene la intención de servir como un reemplazo directo del conjunto de datos original de MNIST para la evaluación comparativa de algoritmos de aprendizaje automático en ejemplos antagónicos. Comparte el mismo tamaño de imagen y la misma estructura de las divisiones de entrenamiento y prueba.

El conjunto de datos original de MNIST contiene una gran cantidad de dígitos escritos a mano. A los miembros de la comunidad de IA/ML/Data Science les encanta este conjunto de datos y lo utilizan como punto de referencia para validar sus algoritmos. De hecho, MNIST suele ser el primer conjunto de datos que intentan los investigadores. "Si no funciona en el MNIST, no funcionará en absoluto", dijeron. "Bueno, si funciona en el MNIST, aún puede fallar en otros".

Compruebe la precisión de sus modelos en este conjunto de datos y mejore la solidez adversaria de los modelos

#### Contenido

Cada imagen tiene 28 píxeles de alto y 28 píxeles de ancho, para un total de 784 píxeles en total. Cada píxel tiene un único valor de píxel asociado, que indica la claridad u oscuridad de ese píxel, y los números más altos significan más oscuro. Este valor de píxel es un número entero entre 0 y 255. Los conjuntos de datos de entrenamiento y prueba tienen 785 columnas. La primera columna consta de las etiquetas de clase (ver arriba) y representa un número entero. El resto de las columnas contienen los valores de píxel de la imagen asociada.

Para ubicar un píxel en la imagen, supongamos que hemos descompuesto x como x = i * 28 + j, donde i y j son números enteros entre 0 y 27. El píxel se encuentra en la fila i y la columna j de una matriz de 28 x 28.
Por ejemplo, 31 indica el píxel que está en la cuarta columna desde la izquierda y la segunda fila desde la parte superior, como en el diagrama ascii a continuación.

#### Descripción

. De las columnas: Cada fila es una imagen

. independiente, la columna 1 es la etiqueta de la clase (enteros del 0 al 9).

. Las columnas restantes son números de píxeles (784 en total).

. Cada valor es la oscuridad del píxel (de 1 a 255)

## Importación de las Librerias

In [ ]:

# Plotting library
from matplotlib import pyplot

# se utiliza para el manejo de rutas y directorios.
import os

# Calculo cientifico y vectorial para python
import numpy as np

# Librerias para graficar
import matplotlib.pyplot as plt

import pandas as pd

# Modulo de optimización de scipy
from scipy import optimize

#Para separa el Dataset 20% y 80% para diferentes pruebas
from sklearn.model_selection import train_test_split

# para aumentar datos en un dataset
from collections import Counter
from imblearn.over_sampling import SMOTE

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

# tells matplotlib to embed plots within the notebook
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1.1 Obtención y Preparación de Datos:

In [ ]:
# Obtenemos los datos 
df_train = pd.read_csv('./mnist_train.csv')
df_test = pd.read_csv('./mnist_test.csv')

# Configurar Pandas para que no corte la visualización
pd.set_option('display.max_rows', 50)  # Mostrar todas las filas (60 -> None)
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas (20 -> None)

# Mostramos los datos de entrenamiento
df_train

In [ ]:
# Datos de entrenamiento del 80% 
X_train = df_train.drop('label', axis=1)
y_train = df_train['label']


# Datos del prueba del 20 % 
X_test = df_test.drop('label', axis=1)
y_test = df_test['label']

# Datos de X_train para el entrenamiento 
print("Datos de X_train")
print(X_train)

print('=' * 100)

print("Datos de y_train")
print(y_train)

# Mostramos la cantidad de ejemplos que se utilizaran para el entrenamiento
print('=' * 100)
print("Cantidad de ejemplos del 80% para el entrenamiento es de: {:.0f}".format(len(X_train)))
print("Cantidad de ejemplos del 20% para la prueba es de: {:.0f}".format(len(X_test)))

In [ ]:
# Mostramos cuantas clases tinene la columna de "Y" labels
df_train['label'].value_counts()

In [ ]:
# Mostramos los datos de entrenamiento X_train
print("Datos de entrenamiento")
X_train

In [ ]:
# Imprimos todas las clases o labels que contiene la columna de y_train
from collections import Counter

num_clases = len(np.unique(y_train))
print("Número de clases:", num_clases)


# Conteo de datos por clase
conteo_clases = dict(Counter(y_train))

# Imprimir el conteo de datos por clase
print("Clases podrían ser:")
for clase, conteo in conteo_clases.items():
    print(f"{clase} : {conteo} datos")

## **2. Visualización de los Datos**

In [ ]:

def displayData(X, example_width=None, figsize=(10, 10)):
    """
    Muestra datos 2D almacenados en X en una bonita cuadrícula.
    """
    # Compute rows, cols
    if X.ndim == 2:
        m, n = X.shape
    elif X.ndim == 1:
        n = X.size
        m = 1
        X = X[None]  # Promocionar a una matriz bidimensional
    else:
        raise IndexError('Input X should be 1 or 2 dimensional.')

    example_width = example_width or int(np.round(np.sqrt(n)))
    example_height = n // example_width  # Cambié esto a división entera

    # Compute number of items to display
    display_rows = int(np.floor(np.sqrt(m)))
    display_cols = int(np.ceil(m / display_rows))

    fig, ax_array = plt.subplots(display_rows, display_cols, figsize=figsize)
    fig.subplots_adjust(wspace=0.025, hspace=0.025)

    ax_array = [ax_array] if m == 1 else ax_array.ravel()

    for i, ax in enumerate(ax_array):
        # Display Image
        h = ax.imshow(X.iloc[i].values.reshape(example_width, example_width),
                      cmap='Greys', extent=[0, 1, 0, 1])
        ax.axis('off')


In [ ]:
# Número de ejemplos de entrenamiento
m = y_train.size
# Se seleccionan 100 datos para ser visualizados
rand_indices = np.random.choice(m, 100, replace=False)
sel = X_train.iloc[rand_indices, :]  # Corregí aquí utilizando iloc
displayData(sel)
plt.show()

In [ ]:
# Visualizar algunas imágenes
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(X_train.iloc[i].values.reshape(28, 28), cmap='gray')
    plt.title(f'Label: {y_train.iloc[i]}')
    plt.axis('off')
plt.show()

## **3. Construcción del Modelo de Red Neuronal**

### 3.1 Definición de Capas

La red neuronal tiene 3 capas: una capa de entrada, una capa oculta y una capa de salida. Las entradas son valores de píxeles de digitos de imagenes. Dado que las imágenes tienen un tamaño de $28 \times 28$, esto nos da 784 unidades de capa de entrada (sin contar la unidad de oscilación adicional que siempre genera +1). Los datos de entrenamiento se cargaron en las variables `X` y `y` anteriores.

Los parámetros tienen dimensiones que están dimensionadas para una red neuronal con 25 unidades en la segunda capa y 10 unidades de salida (correspondientes a las clases de 10 dígitos ya que van de 0 a 9).

In [ ]:
print("X_train shape:", X_train.shape)

In [ ]:
# Contrucción del modelo de red neuronal
# Configuración de parámetros necesario
input_layer_size  = 784  # Entrada de imagenes de digitos de 28x28, caracteristicas
hidden_layer_size = 25   # 25 hidden units, neuronas ocultas, capa uculta
num_labels = 10         # 10 etiquetas, labels del 0 al 9


# carga los pesos en las variables Theta1 y Theta2
pesos = {}
pesos['Theta1'] = np.random.rand(hidden_layer_size, input_layer_size + 1) # 25, 785
pesos['Theta2'] = np.random.rand(num_labels, hidden_layer_size + 1)  # 10, 26

Theta1 = pesos['Theta1']
Theta2 = pesos['Theta2']

# Desenrollar parámetros
print(Theta1.ravel().shape)  # ravel() para convertir los pesos de una matriz a un vector
print(Theta2.ravel().shape)

nn_params = np.concatenate([Theta1.ravel(), Theta2.ravel()])
print(nn_params.shape)

### 3.2 Función de Activación

La principal diferencia es que, en el modelo de `regresión logísitca`, utilizaremos una función de activación conocida como `Sigmoid`.

$$ \sigma(z) = \frac{1}{1 + e^{-z}} $$

In [ ]:
# Función de activación 
def sigmoid(z):
    """
    Computes the sigmoid of z.
    """
    return 1.0 / (1.0 + np.exp(-z))


def sigmoidGradient(z):

    g = np.zeros(z.shape)

    g = sigmoid(z) * (1 - sigmoid(z))

    return g

### 3.3 Función de Pérdida (Costo) y Descenso del Gradiente

Podríamos intentar entrenar nuestro modelo de `regresión logísitca multiclase` con la función de pérdida que ya conocemos. Sin embargo. Esta función es conocida como *Cross Entropy*.

Ahora se implementa la funcion de costo y gradiente para la red neuronal `nnCostFunction`.

La función de costo para la red neuronal (con regularización) es:

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^{m}\sum_{k=1}^{K} \left[ - y_k^{(i)} \log \left( \left( h_\theta \left( x^{(i)} \right) \right)_k \right) - \left( 1 - y_k^{(i)} \right) \log \left( 1 - \left( h_\theta \left( x^{(i)} \right) \right)_k \right) \right] + \frac{\lambda}{2 m} \left[ \sum_{j=1}^{25} \sum_{k=1}^{400} \left( \Theta_{j,k}^{(1)} \right)^2 + \sum_{j=1}^{10} \sum_{k=1}^{25} \left( \Theta_{j,k}^{(2)} \right)^2 \right] $$

In [ ]:
# Definimos la función 
# Esta función calcula la función de costo y los gradientes para una red neuronal de dos capas (una capa oculta) 
# utilizada en un problema de regresión logística multiclase.

def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_):
    
    # Reformar nn_params nuevamente en los parámetros Theta1 y Theta2, las matrices de peso
    # para nuestra red neuronal de 2 capas
    
    #? Los primeros 150 elementos de nn_params se utilizan para llenar la matriz Theta1, 
    # que tendrá dimensiones 25 x 785 (25 filas y 785 columnas).
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))
    
    #? Los elementos apartir de 155 de nn_params se utilizan para llenar la matriz Theta2, 
    # entonces 260 elementos que tendrá dimensiones 10 x 26 (10 filas y 26 columnas.
    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))
    
    m = y.size
    
    J = 0
    
    #? Inicializamos matrices para almacenar los gradientes
    Theta2_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)
    
    a1 = np.concatenate([np.ones((m, 1)), X], axis=1)
    
    a2 = sigmoid(a1.dot(Theta1.T))
    a2 = np.concatenate([np.ones((a2.shape[0], 1)), a2], axis=1)
    
    a3 = sigmoid(a2.dot(Theta2.T))
    
    y_matrix = y.reshape(-1)
    y_matrix = np.eye(num_labels)[y_matrix]
    
    temp1 = Theta1
    temp2 = Theta2
    
    
    #? Calculo de la función de costo con termino de regularización
    
    regularizacion = (lambda_ / (2 * m)) * (np.sum(np.square(temp1[:, 1:])) + np.sum(np.square(temp2[:, 1:])))
    
    J = (-1 / m) * np.sum((y_matrix * np.log(a3)) + (1 - y_matrix) * np.log(1 - a3)) + regularizacion
    
    
    #? Backpropagation
    
    # calcula el error en la capa de salida.
    delta_3 = a3 - y_matrix 
    # calcula el error en la capa oculta propagando hacia atrás el error desde la capa de salida.
    delta_2 = delta_3.dot(Theta2)[:, 1:] * sigmoidGradient(a1.dot(Theta1.T))
    
    # Acumuladores de los gradientes de los pesos Theta1 y Theta2 respectivamente.
    Delta1 = delta_2.T.dot(a1)
    Delta2 = delta_3.T.dot(a2)
    
    # Agregar regularización al gradiente
    
    Theta1_grad = (1 / m) * Delta1
    Theta1_grad[:, 1:] = Theta1_grad[:, 1:] + (lambda_ / m) * Theta1[:, 1:]
    
    Theta2_grad = (1 / m) * Delta2
    Theta2_grad[:, 1:] = Theta2_grad[:, 1:] + (lambda_ / m) * Theta2[:, 1:]
    
    # Toodas las gradientes
    
    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])
    
    return J, grad


#### Pruebando el funcionamiento de la función de costo 

In [ ]:
lambda_ = 0.1

J, _ = nnCostFunction(nn_params, input_layer_size, hidden_layer_size,  num_labels, X_train, y_train.values, lambda_)
print("Costo en parámetros (Cargado): %.f " % J)
print('El costo debe esta cercano a          : 0.287629')

### Inicializamso la función para inicializar los pesos 

La función randInitializeWeights se utiliza para inicializar aleatoriamente los pesos de una capa en una red neuronal. Esta inicialización aleatoria es crucial para evitar que todos los pesos comiencen con el mismo valor y se sincronicen durante el entrenamiento, lo que puede llevar a problemas de simetría y convergencia inadecuada durante el aprendizaje.

In [ ]:
# Inicializamos los pesos de la red neuronal de forma aleatoria utilizando la función randInitializeWeights
def randInitializeWeights(L_in, L_out, epsilon_init = 0.12):
    
    """
        Inicializa aleatoriamente los pesos de una capa en una red neuronal.

        Parámetros
        ----------
        L_in:int
        Número de conexiones entrantes.

        L_salida: int
        Número de conexiones salientes.

        epsilon_init: flotante, opcional
        Rango de valores que puede tomar el peso de un uniforme
        distribución.

        Devoluciones
        -------
        W: tipo matriz
        El peso inicializado a valores aleatorios. Tenga en cuenta que W debería
        establecerse en una matriz de tamaño (L_out, 1 + L_in) como
        la primera columna de W maneja los términos de "sesgo".
    """
    
    """
    . Se crea una matriz W de dimensiones (L_out, 1 + L_in) inicializada con ceros.
    . L_out: representa el número de neuronas en la capa actual (conexiones salientes).
    . 1 + L_in: se refiere al número de entradas a cada neurona, incluyendo un término de "sesgo" (bias) que corresponde a la primera columna de la matriz W.
    """
    
    # Creamos una matriz con las dimensiones 
    W = np.zeros((L_out, 1 + L_in))
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init
    
    return W

In [ ]:
# Llamamos a la funcion para inicializar los thetas aleatoriamente
print("Inicialización de parámetros de redes neuronales...")

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

# Desenrrollar parámetos
initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)

## 3.4 Backpropagation

Ahora, se implementará el algoritmo de retropropagación. Recuerde que la intuición detrás del algoritmo de retropropagación es la siguiente. Dado un ejemplo de entrenamiento $(x^{(t)}, y^{(t)})$, primero ejecutaremos un "pase hacia adelante" para calcular todas las activaciones en toda la red, incluido el valor de salida de la hipótesis $h_\theta(x)$. Luego, para cada nodo $j$ en la capa $l$, se busca calcular un "término de error" $\delta_j^{(l)}$ que mide cuánto ese nodo fue "responsable" de cualquier error en la salida.

### 3.4  Comprobación del gradiente

En la red neuronal, se está minimizando la función de costo $J(\Theta)$. Para realizar una verificación de gradiente en sus parámetros, se puede imaginar "desenrollar" los parámetros $\Theta^{(1)}$, $\Theta^{(2)}$ en un vector largo $\theta$. Al hacerlo, se puede pensar que la función de costo es $J(\Theta)$ y usar el siguiente procedimiento de verificación de gradiente.


** Consejo práctico **: al realizar la verificación de gradientes, es mucho más eficiente utilizar una pequeña red neuronal con un número relativamente pequeño de unidades de entrada y unidades ocultas, por lo que tiene un número relativamente pequeño
de parámetros. Cada dimensión de $\theta$ requiere dos evaluaciones de la función de costo y esto puede resultar costoso. En la función `checkNNGradients`, nuestro código crea un pequeño modelo aleatorio y un conjunto de datos que se usa con `computeNumericalGradient` para verificar el gradiente. Además, una vez que esté seguro de que sus cálculos de gradiente son correctos, debe desactivar la verificación de gradiente antes de ejecutar su algoritmo de aprendizaje.


** Sugerencia práctica: ** La verificación del gradiente funciona para cualquier función en la que esté calculando el costo y el gradiente. Concretamente, puede usar la misma función `computeNumericalGradient` para verificar si sus implementaciones de gradiente para los otros ejercicios también son correctas (por ejemplo, la función de costo de regresión logística).

In [ ]:
#? Function para depurar Inicializar pesos
def debugInitializeWeights(fan_out, fan_in):
    """
    Inicializar los pesos de una capa con conexiones entrantes fan_in y salidas fan_out
    conexiones usando una estrategia fija. Esto le ayudará más adelante en la depuración.

    Tenga en cuenta que W debe establecerse como una matriz de tamaño (1+fan_in, fan_out) como la primera fila de W maneja
    los términos de "sesgo".

    Parámetros
    ----------
    fan_out: int
    El número de conexiones salientes.

    fan_in: int
    El número de conexiones entrantes.

    Devoluciones
    -------
    W: array_like (1+fan_in, fan_out)
    La matriz de pesos inicializada dadas las dimensiones.
    """
    # Initialize W using "sin". This ensures that W is always of the same values and will be
    # useful for debugging
    W = np.sin(np.arange(1, 1 + (1+fan_in)*fan_out))/10.0
    W = W.reshape(fan_out, 1+fan_in, order='F')
    
    return W


#? Función para calcular gradiente numérico
def computeNumericalGradient(J, theta, e=1e-4):
    """
    Calcula el gradiente usando "diferencias finitas" y nos da una estimación numérica del
    degradado.

    Parámetros
    ----------
    J: función
    La función de costo que se utilizará para estimar su gradiente numérico.

    theta: tipo matriz
    Los parámetros de red unidimensionales desenrollados. El gradiente numérico se calcula en
    esos parámetros dados.

    e: flotante (opcional)
    El valor que se utilizará para épsilon para calcular la diferencia finita.

    Notas
    -----
    El siguiente código implementa la verificación de gradiente numérico y
    devuelve el gradiente numérico. Establece `numgrad[i]` en (un valor numérico
    aproximación de) la derivada parcial de J con respecto a la
    i-ésimo argumento de entrada, evaluado en theta. (es decir, `numgrad[i]` debería
    ser (aproximadamente) la derivada parcial de J con respecto
    a theta[i].)
    """
    numgrad = np.zeros(theta.shape)
    perturb = np.diag(e * np.ones(theta.shape))
    for i in range(theta.size):
        loss1, _ = J(theta - perturb[:, i])
        loss2, _ = J(theta + perturb[:, i])
        numgrad[i] = (loss2 - loss1)/(2*e)
        
    return numgrad


#? Función para comprobar gradientes NN
def checkNNGradients(nnCostFunction, lambda_=0):
    """
    Crea una pequeña red neuronal para comprobar los gradientes de retropropagación. Dará salida al
    gradientes analíticos producidos por su código backprop y los gradientes numéricos
    (calculado usando ComputeNumericalGradient). Estos dos cálculos de gradiente deberían dar como resultado
    valores muy similares.

    Parámetros
    ----------
    nnCostoFunción: func
    Una referencia a la función de costos implementada por el estudiante.

    lambda_: flotante (opcional)
    El valor del parámetro de regularización.
    """
    input_layer_size = 784
    hidden_layer_size = 25
    num_labels = 10
    m = 25

    # We generate some 'random' test data
    Theta1 = debugInitializeWeights(hidden_layer_size, input_layer_size)
    Theta2 = debugInitializeWeights(num_labels, hidden_layer_size)

    # Reusing debugInitializeWeights to generate X
    X = debugInitializeWeights(m, input_layer_size - 1)
    y = np.arange(1, 1+m) % num_labels
    # print(y)
    # Unroll parameters
    nn_params = np.concatenate([Theta1.ravel(), Theta2.ravel()])

    # short hand for cost function
    costFunc = lambda p: nnCostFunction(p, input_layer_size, hidden_layer_size,
                                        num_labels, X, y, lambda_)
    cost, grad = costFunc(nn_params)
    numgrad = computeNumericalGradient(costFunc, nn_params)

    # Visually examine the two gradient computations.The two columns you get should be very similar.
    print(np.stack([numgrad, grad], axis=1))
    print('Las dos columnas anteriores que obtenga deberían ser muy similares.')
    print('(Izquierda: su gradiente numérico, gradiente analítico derecho)\n')

    # Evaluate the norm of the difference between two the solutions. If you have a correct
    # implementation, and assuming you used e = 0.0001 in computeNumericalGradient, then diff
    # should be less than 1e-9.
    diff = np.linalg.norm(numgrad - grad)/np.linalg.norm(numgrad + grad)

    print('Si su implementación de retropropagación es correcta, entonces \n'
          'la diferencia relativa será pequeña (menos de 1e-9).. \n'
          'Diferencia relativa: %g' % diff)


In [ ]:
# Realizamos la prueba de la función 
checkNNGradients(nnCostFunction)

Un ejemplo de forma regularizada

In [ ]:
# Verificamos los gradientes ejecutando checkNNGradients
lambda_ = 3
checkNNGradients(nnCostFunction, lambda_)

# También genera los valores de depuración de costFunction
debug_J, _  = nnCostFunction(nn_params, input_layer_size,
                          hidden_layer_size, num_labels, X_train, y_train.values, lambda_)

print('\n\nCosto en parámetros de depuración (fijos) (w/ lambda = %f): %f ' % (lambda_, debug_J))
print('(for lambda = 3, this value should be about 0.576051)')

## **4. Entrenamiento del Modelo**

In [ ]:
# Entrenamiento del modelo 

# Después de haber completado la tarea, cambie el maxiter a uno más grande
#? valor para ver cómo ayuda más formación.
# Especifica el número máximo de iteraciones que el algoritmo de optimización 
options = {'maxiter': 100}

#? Probar con diferentes valores de lambda
lambda_ = 1

#? Cree una "taquigrafía" para minimizar la función de costos.
#? Ahora, costFunction es una función que toma solo un argumento.
costFunction = lambda p: nnCostFunction(p, input_layer_size, 
                                        hidden_layer_size, 
                                        num_labels, X_train, y_train, lambda_)

#? Ahora, costFunction es una función que toma solo un argumento.
#? (los parámetros de la red neuronal)

res = optimize.minimize(costFunction,
                        initial_nn_params,
                        jac=True,
                        method='TNC',
                        options=options)

#? obtenemos la solución de la optimización
nn_params = res.x

#? Obtenemos Theta1 y Theta2 de nn_params
Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))

Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))